In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE

In [ ]:
# Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

# To display all the columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# To map Empty Strings or numpy.inf as Na Values
pd.options.mode.use_inf_as_na = True

%matplotlib inline

# Set Style
sns.set(style = "whitegrid")

In [ ]:
pd.set_option('display.max_columns',None)

# <font color=Indigo>Analysis Process</font>
The analysis is divided into nine main parts:
> 1. Data Sourcing or Data Understanding
2. Data cleaning and Derived Metrics
3. Exploratory Data Analysis (Univariate, Bivariate Analysis)
4. Model Preparation
     - Train and Test Data Split
     - Data Normalization
     - Handling Class Imbalance
5. Model Building
6. Residual Analysis of the Train Data
7. Making Predictions
8. Model Evaluation
9. Final Inference

# 1.Data import

In [ ]:
telecom = pd.read_csv('telecom_churn_data.csv')

In [ ]:
telecom.head()

In [ ]:
telecom.shape

In [ ]:
telecom.describe()

In [ ]:
telecom.info()

In [ ]:
# Lets check the percentage of missing values in each column
(telecom.isnull().sum()/ len(telecom)) * 100

Inference-from the above percentages we see that there are a few columns which have more than 70% of data missing. We can drop these columns as they would convey a very little information 

We should not drop the columns related to the recharge amounts as they would tell us who are high value customers. Thus to avoid doing that first we have to deal with the missing values in these columns -  

Here is that we have to define **high-value customers** as - **Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase)**.

- total_rech_data_6  - 74.8467 % missing values
- total_rech_data_7  - 74.4287 % missing values
- av_rech_amt_data_6 - 74.8467 % missing values
- av_rech_amt_data_7 - 74.4287 % missing values

We have 2 more columns as -

- date_of_last_rech_data_6 - 74.8467 % missing values
- date_of_last_rech_data_7 - 74.4287 % missing values


We are dealing with users churn - where Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time.


# Part 2. Data Preparation

## 2.1 Duplicate check

In [ ]:
telecom['mobile_number'].duplicated().sum()

No duplicates

In [ ]:
telecom.set_index('mobile_number',inplace = True)

## 2.2 Derivied columns

#### 1.Total rech columns

As we have values for 'total_rech_data_' and 'av_rech_amt_data_' for the particular months 6, 7, 8 & 9. Using these 2 values we can derive new feature for the respective months called total_data_rech_amt_ which equals
total_rech_data_ * av_rech_amt_data_

This new feature would help in computing the total data recharge amount - 'total_data_rech_amt_' for the months 6, 7, 8 & 9.

In [ ]:
# If no recharge is done for the month then 'total_rech_data_6' and 'date_of_last_rech_data_6' both will have null values in the
# same row. So, if index match for record, we can impute the 'total_rech_data_6' and 'av_rech_amt_data_6' with 0. Same for the
# other two months as well.

total_recharge_6_index = telecom['total_rech_data_6'].isnull()
date_of_last_recharge_6_index = telecom['date_of_last_rech_data_6'].isnull()

if total_recharge_6_index.equals(date_of_last_recharge_6_index):
    print(' NULL values for month 6 are imputed')
    telecom['total_rech_data_6'].fillna(0, inplace=True)
    telecom['av_rech_amt_data_6'].fillna(0, inplace=True)

total_rech_7_index = telecom['total_rech_data_7'].isnull()
date_of_last_rech_7_index = telecom['date_of_last_rech_data_7'].isnull()

if total_rech_7_index.equals(date_of_last_rech_7_index):
    print(' NULL values for month 7 are imputed')
    telecom['total_rech_data_7'].fillna(0, inplace=True)
    telecom['av_rech_amt_data_7'].fillna(0, inplace=True)

total_rech_8_index = telecom['total_rech_data_8'].isnull()
date_of_last_rech_8_index = telecom['date_of_last_rech_data_8'].isnull()

if total_rech_8_index.equals(date_of_last_rech_8_index):
    print(' NULL values for month 8 are imputed')
    telecom['total_rech_data_8'].fillna(0, inplace=True)
    telecom['av_rech_amt_data_8'].fillna(0, inplace=True)

Inference- We see that the two indexes object are equal and we can safely conclude that no data recharge was done for months 6, 7 and 8 & the missing values can be imputed with 0. 

In [ ]:
rech_cols = []
for x in telecom.columns.tolist():
    if (('rech' in x) or ('amt' in x) or ('arpu' in x)) and (('date' not in x) and ('day' not in x)):
        rech_cols.append(x)

In [ ]:
telecom[rech_cols].head()

In [ ]:
telecom[rech_cols].describe()

In [ ]:
telecom[['total_rech_num_6','total_rech_amt_6','total_rech_data_6','av_rech_amt_data_6']].head()

In [ ]:
copy = telecom.copy()

In [ ]:
copy[rech_cols].isnull().sum()

from the describe we can see that it is safe to assume to replace null with 0 for all the rech_amt,rech_count and max columns

In [ ]:
# Columns with more than 70% data missing

cols_with_70_percent_missing_data = ((telecom.isnull().sum()/ len(telecom)) * 100) >= 70
cols_with_70_percent_missing_data.sum()

Inference - Our objective is to only find out the churn rate of the High valued customers .

In [ ]:
telecom['total_amt_6'] = copy['total_rech_amt_6']+copy['total_rech_data_6']*copy['av_rech_amt_data_6']
telecom['total_amt_7'] = copy['total_rech_amt_7']+copy['total_rech_data_7']*copy['av_rech_amt_data_7']
telecom['total_amt_8'] = copy['total_rech_amt_8']+copy['total_rech_data_8']*copy['av_rech_amt_data_8']
telecom['total_amt_9'] = copy['total_rech_amt_9']+copy['total_rech_data_9']*copy['av_rech_amt_data_9']

In [ ]:
telecom['total_amt_6_7'] =telecom['total_amt_6']+telecom['total_amt_7']
telecom['total_amt_8_9'] =telecom['total_amt_8']+telecom['total_amt_9']
telecom['total_amt_6_9'] = telecom['total_amt_6']+telecom['total_amt_7'] +telecom['total_amt_8']+telecom['total_amt_9'] 

#### 2. Av amount -

In [ ]:
telecom['total_rech_count_6'] = copy['av_rech_amt_data_6']+copy['total_rech_num_6']
telecom['total_rech_count_7'] = copy['av_rech_amt_data_7']+copy['total_rech_num_7']
telecom['total_rech_count_8'] = copy['av_rech_amt_data_8']+copy['total_rech_num_8']
telecom['total_rech_count_9'] = copy['av_rech_amt_data_9']+copy['total_rech_num_9']

In [ ]:
telecom['av_amt_6_7'] = telecom['total_amt_6_7']/(telecom['total_rech_count_6']+telecom['total_rech_count_7'])

In [ ]:
telecom['av_amt_6_7'].isnull().sum()

In [ ]:
telecom[telecom['av_amt_6_7'].isnull()][['av_amt_6_7','total_rech_count_6','total_rech_count_7','total_amt_6_7']].head()

In [ ]:
# all the rows where the average amount is nan, the total recharges are 0.Therefore replacing null with 0
telecom['av_amt_6_7'] = telecom['av_amt_6_7'].replace(np.nan,0) 

In [ ]:
telecom['av_amt_6_7'].isnull().sum()

#### 3 High value customers
As we are concerned with high-value customers. Define high-value customers as those customers who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase).


Those customers who have spend an average amount more than the 70 percentile of average amount in the first two months are considered as HVCs.

In [ ]:
telecom_hvc = telecom[telecom['av_amt_6_7']>telecom['av_amt_6_7'].quantile(0.7)]

In [ ]:
telecom_hvc.shape

Inference-So after filtering out the high-value customers we are left with 29993 observations. The shape of the telecom dataset is now (29993, 237)

#### 4 Tagging customers who have Churned-

Those customers who have not made any calls or received any calls and not have used any form of mobile data are considered as churned customers

#### null value check for those 4 columns

In [ ]:
telecom_hvc[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].isnull().sum()

In [ ]:
telecom_hvc[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].head()

taking the maximum among the 4 columns and equating it to 0 to identify the churn customers. Single the max value for such customers will be 0

In [ ]:
churn_indices = telecom_hvc[telecom_hvc[['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].max(axis =1) == 0].index

In [ ]:
print(len(churn_indices))

In [ ]:
telecom_hvc.loc[churn_indices,['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']].head()

In [ ]:
telecom_hvc['churn'] = 0

In [ ]:
telecom_hvc.loc[churn_indices,'churn'] = 1

In [ ]:
telecom_hvc['churn'].value_counts()

# Part 3 - Data Cleaning & Missing Values Treatment

### 1. Columns

In [ ]:
(telecom.isnull().sum(axis = 0)/ len(telecom)) * 100

### Dropping all columns corresponding to churn phase

In [ ]:
# columns that have more than 40% data missing.
total_cols_with_atleast_40_percent_missing_data = ((telecom.isnull().sum()/ len(telecom)) * 100) >= 40
total_cols_with_atleast_40_percent_missing_data.sum()

In [ ]:
# Listing the column/feature names which have more than 40% missing data.

cols_names_with_40_percent_missing_value = total_cols_with_atleast_40_percent_missing_data[
    total_cols_with_atleast_40_percent_missing_data > 0.40].index

cols_names_with_40_percent_missing_value

### Now lets examine the above columns (with >40% missing values) one by one -

In [ ]:
print("Total NULL values for max_rech_data_6 columns is - ", telecom.max_rech_data_6.isnull().sum())
print("Total NULL values for max_rech_data_7 columns is - ", telecom.max_rech_data_7.isnull().sum())
print("Total NULL values for max_rech_data_8 columns is - ", telecom.max_rech_data_8.isnull().sum())

Inference - Missing values can be infered as No recharge was done and can be imputed with 0 for all the 3 columns.

In [ ]:
# Replace NA values with 0
for col in ['max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8']:
    telecom[col].fillna(0, inplace=True)

Now all the columns that have more than 40% missing values one way or the other relate to data/mobile internet usage for the months 6, 7 & 8 respectively. One important observation that comes out is that for a particular month all these values have same number of missing values. The below data tells us the same.

In [ ]:
churn_phase_cols = []
for x in telecom_hvc.columns.tolist():
    if '_9' in x:
        churn_phase_cols.append(x)

In [ ]:
print(churn_phase_cols)

In [ ]:
print(len(churn_phase_cols))

In [ ]:
telecom_hvc_1 =  telecom_hvc.drop(churn_phase_cols,axis = 1)

In [ ]:
print(telecom_hvc.shape[1]-telecom_hvc_1.shape[1])

In [ ]:
telecom_hvc_1.info()

In [ ]:
telecom_hvc_1.head()

### Null value check

In [ ]:
null_columns = (telecom_hvc_1.isnull().sum()/len(telecom_hvc_1))[telecom_hvc_1.isnull().sum()/len(telecom_hvc_1) > 0]

In [ ]:
print(len(null_columns))

In [ ]:
# converting into percentage
null_columns = null_columns*100

In [ ]:
null_columns.sort_values(ascending = False)

In [ ]:
null_columns[null_columns>20]

In [ ]:
print(len(null_columns[null_columns>20]))

In [ ]:
telecom_2 = telecom_hvc_1.drop(null_columns[null_columns>20].index.tolist(),axis = 1)

In [ ]:
print(telecom_hvc_1.shape[1]-telecom_2.shape[1])

In [ ]:
telecom_2.info()

In [ ]:
null_columns.drop(null_columns[null_columns>20].index.tolist(),inplace = True)

In [ ]:
print(null_columns.sort_values(ascending = False))

all the columns have less than 6% of null values.Lets check if there any rows which contain large number of null values

In [ ]:
null_values_rows = telecom_2.isnull().sum(axis = 1)

In [ ]:
null_values_rows[null_values_rows > 0].value_counts()

In [ ]:
telecom_2.loc[null_values_rows[null_values_rows > 0].index,'churn'].value_counts()

__There rows with null values have about 30% of total churn customers. Therefore dropping these rows will result in significant loss of churn data.Therefore we have to impute these values instead of dropping__

### Imputing Null values in the rows

In [ ]:
telecom_2.info()

In [ ]:
telecom_2.select_dtypes(include = 'object').columns

In [ ]:
# all are date columns hence converting them into datetime
for column in telecom_2.select_dtypes(include = 'object').columns:
    telecom_2[column] = telecom_2[column].astype('datetime64[ns]')

In [ ]:
telecom_2.info()

In [ ]:
telecom_2[null_columns.index].info()

In [ ]:
print(telecom_2['last_date_of_month_6'].value_counts())
print(telecom_2['last_date_of_month_7'].value_counts())
print(telecom_2['last_date_of_month_8'].value_counts())

In [ ]:
# Dropping last date columns as they only signify the last date of month.
telecom_hvc_3 = telecom_2.drop(['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8'],axis = 1)

In [ ]:
print(telecom_2.shape[1]-telecom_hvc_3.shape[1])

In [ ]:
# Date_of_last_rech columns
# creating new columns called is_rech for each month which shows if a customer has recharged in the particular month or not.
telecom_hvc_3['is_rech_6'] = 1
telecom_hvc_3['is_rech_7'] = 1
telecom_hvc_3['is_rech_8'] = 1

telecom_hvc_3.loc[telecom_hvc_3['date_of_last_rech_6'].isnull(),'is_rech_6'] = 0
telecom_hvc_3.loc[telecom_hvc_3['date_of_last_rech_7'].isnull(),'is_rech_7'] = 0
telecom_hvc_3.loc[telecom_hvc_3['date_of_last_rech_8'].isnull(),'is_rech_8'] = 0

In [ ]:
# Dropping  Date_of_last_rech columns as they only signify the last date of month.
telecom_4 = telecom_hvc_3.drop(['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8'],axis = 1)

In [ ]:
print(telecom_hvc_3.shape[1]-telecom_4.shape[1])

In [ ]:
null_columns.drop(['date_of_last_rech_6','date_of_last_rech_7','date_of_last_rech_8','last_date_of_month_7','last_date_of_month_8'],inplace = True)

In [ ]:
telecom_4[null_columns.index].describe()

## Insight 1-

Inference-We can also see an observation with mobile internet usage having a missing value would mostly mean that the customer is not using that particular service. This could also mean that the person would not be using any other kind of  add-on services that would require a mobile internet pack.
Also with this inference we can impute the missing values related to columns for mobile data with 0.


In [ ]:
# Replacing all null values with 0
telecom_hvc_5 = telecom_4.replace(np.nan,0)

In [ ]:
print(telecom_hvc_5.isnull().sum().sum())

In [ ]:
telecom_hvc_5.info()

### Dropping columns with single value in the entire column

In [ ]:
#creating a dictionary with column and number of unique values
unique_value_dict = {}
for x in telecom_hvc_5.columns.tolist():
    l = len(telecom_hvc_5[x].value_counts())
    unique_value_dict[x] = l

In [ ]:
unique_df = pd.DataFrame.from_dict(unique_value_dict,orient = 'index')

In [ ]:
unique_df[unique_df[0] == 1].index

In [ ]:
# Dropping all the columns with only one unique value
telecom_hvc_6 = telecom_hvc_5.drop(unique_df[unique_df[0] == 1].index,axis = 1)

In [ ]:
telecom_hvc_6.info()

### Dropping columns with high percentage of single value

In [ ]:
#creating a dictionary with column and number of unique values
high_value_dict = {}
for x in telecom_hvc_6.columns.tolist():
    l = round(100*(telecom_hvc_6[x].value_counts().iloc[0]/len(telecom_hvc_6)))
    high_value_dict[x] = l

In [ ]:
high_df = pd.DataFrame.from_dict(high_value_dict,orient = 'index')

In [ ]:
high_df.sort_values(by = 0,ascending = False).head(15)

In [ ]:
# Dropping all the columns with high percentage of single value
telecom_hvc_6 = telecom_hvc_6.drop(high_df[high_df[0] >= 99].index,axis = 1)

In [ ]:
telecom_hvc_6.info()

#### Renaming month name :

In [ ]:
telecom_hvc_6.rename(columns = {'aug_vbc_3g':'vbc_3g_8','jun_vbc_3g':'vbc_3g_6','jul_vbc_3g':'vbc_3g_7','sep_vbc_3g':'vbc_3g_9'},inplace = True)

In [ ]:
telecom_hvc_6.drop('vbc_3g_9',axis =1,inplace = True)

In [ ]:
telecom_hvc_6.info()

#### Check for zero variance -
Let's check on the other columns now. But before proceeding with the missing value analysis of other columns let's check if we have any columns with zero variance (as this would be very important while conducting **Principal Component Analysis**). For the problem at hand if any variable has zero variance (all same values) it would have no impact during the analysis. We will drop duch columns.

We can use var() for this.

In [ ]:
 # Variance  - df.var() method
columns_with_0_variance = telecom.var() == 0
print("Total columns with ZERO variance are - ", columns_with_0_variance.sum())
column_name_with_0_variance = columns_with_0_variance[columns_with_0_variance == 1].index
print(column_name_with_0_variance)

In [ ]:
#Checking the column for missing values

In [ ]:
columns_with_null_values = telecom.columns[telecom.isna().any()].tolist()
print(columns_with_null_values)

In [ ]:
# Month 6
cols_with_null_values_for_month_6 = [col for col in columns_with_null_values if '_6' in col]
print(telecom[cols_with_null_values_for_month_6].info())

# Month 7
cols_with_null_values_for_month_7 = [col for col in columns_with_null_values if '_7' in col]
print(telecom[cols_with_null_values_for_month_7].info())

# Month 8
cols_with_null_values_for_month_8 = [col for col in columns_with_null_values if '_8' in col]
print(telecom[cols_with_null_values_for_month_8].info())

Inference- . On mapping these attributes using the data dictionary we know that all these attributes point to the minutes of usage calls (incoming/outgoing) made within the network or to outside network.

So if any value is missing means that the minutes of usage is missing and can be imputed with 0.

The same would apply for the columns pointing to months 7 & 8 aswell.

## Insight 2-

Also one more insight that the above figures give us is that towards the 8th month the number of missing values increases and this could possibly mean that the customer might have stopped using these services. This could be an indication that the particular customer is more likely to churn.

So not using the services means the minutes of usage can be imputed with 0.

In [ ]:
# Impute missing values with 0 excluding the date columns.
for column in columns_with_null_values:
    if "date_of_last_rech" not in column:
        telecom[column].fillna(0, inplace=True)

In [ ]:
# description for missing values-
print(telecom.isnull().sum())

# Insight 3 -

If the index of missing values are equal then we can assume that they are for same record

In [ ]:
# If the index are equal for all entries, then we can confirm that the missing values are all from the same observations/index.

# Month 6
arpu_3g_6_index = telecom['arpu_3g_6'].isnull()
night_pck_user_6_index = telecom['night_pck_user_6'].isnull()
count_rech_2g_6_index = telecom['count_rech_2g_6'].isnull()
count_rech_3g_6_index = telecom['count_rech_3g_6'].isnull()
fb_user_6_index = telecom['fb_user_6'].isnull()

if arpu_3g_6_index.equals(night_pck_user_6_index) & night_pck_user_6_index.equals(count_rech_2g_6_index) & count_rech_2g_6_index.equals(count_rech_3g_6_index) &  count_rech_3g_6_index.equals(fb_user_6_index):
    print('The indexes for NULL values for month 6 are equal')

# Month 7
arpu_3g_7_index = telecom['arpu_3g_7'].isnull()
night_pck_user_7_index = telecom['night_pck_user_7'].isnull()
count_rech_2g_7_index = telecom['count_rech_2g_7'].isnull()
count_rech_3g_7_index = telecom['count_rech_3g_7'].isnull()
fb_user_7_index = telecom['fb_user_7'].isnull()

if arpu_3g_7_index.equals(night_pck_user_7_index) & night_pck_user_7_index.equals(count_rech_2g_7_index) & count_rech_2g_7_index.equals(count_rech_3g_7_index) &  count_rech_3g_7_index.equals(fb_user_7_index):
    print('The indexes for NULL values for month 7 are equal')

# Month 8
arpu_3g_8_index = telecom['arpu_3g_8'].isnull()
night_pck_user_8_index = telecom['night_pck_user_8'].isnull()
count_rech_2g_8_index = telecom['count_rech_2g_8'].isnull()
count_rech_3g_8_index = telecom['count_rech_3g_8'].isnull()
fb_user_8_index = telecom['fb_user_8'].isnull()

if arpu_3g_8_index.equals(night_pck_user_8_index) & night_pck_user_8_index.equals(count_rech_2g_8_index) & count_rech_2g_8_index.equals(count_rech_3g_8_index) &  count_rech_3g_8_index.equals(fb_user_8_index):
    print('The indexes for NULL values for month 8 are equal')


Analysing the revenue columns-

In [ ]:
telecom['arpu_6'].describe()

In [ ]:
telecom['arpu_7'].describe()

In [ ]:
telecom['arpu_8'].describe()

Inference-The above statistical description of the ARPU - Average Revenue Per User column gives some important insight. The MINIMUM value for all the arpu related columns are NEGATIVE. Now as per the defination of ARPU which says -

"The average revenue per user is the average billing per customer earned by the telecom company every month".Now the revenue generated from a user cannot be negative. If a customer is not using any services then apru for the person would be zero (rather that being negative). Now if arpu is negative for any row, then that would mean that is a wrong/corrupt data. It will be of no use to us for analysis.

In [ ]:
# Index where the arpu values for month 6 are less than 0 -

arpu_6_index = (telecom['arpu_6'] < 0)
print('Total observations with negative arpu values for month 6 -', arpu_6_index.sum())

# Index where the arpu values for month 7 are less than 0 -

arpu_7_index = (telecom['arpu_7'] < 0)
print('Total observations with negative arpu values for month 7 -', arpu_7_index.sum())

# Index where the arpu values for month 8 are less than 0 -

arpu_8_index = (telecom['arpu_8'] < 0)
print('Total observations with negative arpu values for month 8 -', arpu_8_index.sum())

In [ ]:
# Let's delete the observations with negative arpu values. 

telecom= telecom[(telecom['arpu_6'] >= 0) & (telecom['arpu_7'] >= 0) & (telecom['arpu_8'] >= 0)]

In [ ]:
telecom_hvc_6.shape

We can now go ahead with EDA

# Part 4 - Exploratory Data Analysis

### Categorical column check-

In [ ]:
# Let's check if there are any categorical variables in the dataframe

In [ ]:
telecom.describe()

In [ ]:
category_cols = ['night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8', 'fb_user_6', 'fb_user_7', 'fb_user_8']

telecom[category_cols] =telecom[category_cols].astype(int)

telecom[category_cols].info()

##  Univariate analysis-

#### 1. Univariate Analysis - Ordered Categorical Variables

In [ ]:
#  Function to add data labels in the graph

def def_data_labels(ax, spacing = 5):

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.2f}%".format(y_value)

        # Create annotation
        plt.annotate(
            label,                        # Use `label` as label
            (x_value, y_value),           # Place label at end of the bar
            xytext = (0, space),          # Vertically shift label by `space`
            textcoords = "offset points", # Interpret `xytext` as offset in points
            ha = 'center',                # Horizontally center label
            va = va)           

In [ ]:
# Univariate Plot Analysis of Ordered categorical variables vs Percentage Rate
category_col = ['night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8', 'fb_user_6', 'fb_user_7', 'fb_user_8']
counter = 1

plt.figure(figsize = (15, 12))

for col_list in category_col:
        
    series = round(((telecom[col_list].value_counts(dropna = False))/(len(telecom[col_list])) * 100), 2)

    plt.subplot(2, 3, counter)
    ax = sns.barplot(x = series.index, y = series.values, order = series.sort_index().index)
    plt.xlabel(col_list, labelpad = 15)
    plt.ylabel('Percentage Rate', labelpad = 10)

    # Call Custom Function
    def_data_labels(ax)

    counter += 1

del category_col, counter, ax

plt.subplots_adjust(hspace = 0.3)
plt.subplots_adjust(wspace = 0.5)
plt.show()

Inference -From the `night_pck_user` variables, it can be inferred that almost 99% of users are not using nightly pack in all the 3 months.The number of users using the pack is a very small percentage of users.The % of users not using the pack is slightly going up .

However users are using facebook(fb) `fb_user` in equal percentage and it is observed that as the number of months progress, there is a small decline in the fb usage.

### Churn

In [ ]:
print(telecom_hvc_6.churn.value_counts())
print(100*telecom_hvc_6.churn.value_counts()/len(telecom_hvc_6))

Only 10% of the high value customers are churn customers

### Recharge amount

### Total_amt including data recharge

In [ ]:
telecom_hvc_6[['total_amt_6','total_amt_7','total_amt_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['total_amt_6','total_amt_7','total_amt_8','churn']].median()

Similar to the total rech amount, slight down trend but not too steep of a decline

### total_amt vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['total_amt_6','total_amt_7','total_amt_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['total_amt_6','total_amt_7','total_amt_8','churn']].groupby('churn').median()

Similar trend for the total rech amount as well

##  >> Derived column: difference from consecutive months

In [ ]:
telecom_hvc_6['diff_amt_6_7'] = telecom_hvc_6['total_amt_6'] - telecom_hvc_6['total_amt_7']
telecom_hvc_6['diff_amt_7_8'] = telecom_hvc_6['total_amt_7'] - telecom_hvc_6['total_amt_8']
telecom_hvc_6['diff_amt_6_8'] = telecom_hvc_6['total_amt_6'] - telecom_hvc_6['total_amt_8']

In [ ]:
telecom_hvc_6[['diff_amt_6_7','diff_amt_7_8','diff_amt_6_8','churn']].groupby('churn').mean()

Inference-We can see that in case of the non churn customers the mean of the difference in recharges is negative indicating that the recharge amount is increasing with number of months, where as we can see that there is a huge decrease in case of churn customers

### Total number of recharges

In [ ]:
telecom_hvc_6[['total_rech_count_6','total_rech_count_7','total_rech_count_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['total_rech_count_6','total_rech_count_7','total_rech_count_8']].median()

Inference-More or less same count can be observed through the 3 months.

### total_rech_count vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['total_rech_count_6','total_rech_count_7','total_rech_count_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['total_rech_count_6','total_rech_count_7','total_rech_count_8','churn']].groupby('churn').median()

The number recharges also follow the similar trend of the recharge amounts

## >>  Derived column: difference from consecutive months

In [ ]:
telecom_hvc_6['diff_count_6_7'] = telecom_hvc_6['total_rech_count_6'] - telecom_hvc_6['total_rech_count_7']
telecom_hvc_6['diff_count_7_8'] = telecom_hvc_6['total_rech_count_7'] - telecom_hvc_6['total_rech_count_8']
telecom_hvc_6['diff_count_6_8'] = telecom_hvc_6['total_rech_count_6'] - telecom_hvc_6['total_rech_count_8']

In [ ]:
telecom_hvc_6[['diff_count_6_7','diff_count_7_8','diff_count_6_8','churn']].groupby('churn').mean()

Inference-We can see that in case of the non churn customers the mean of the difference in recharge counts is negative indicating that the recharge count is increasing with number of months, where as we can see that there is a decrease in case of churn customers

## Incoming calls

In [ ]:
[col for col in telecom_hvc_6.columns if 'ic' in col]

### Total_incoming calls

In [ ]:
telecom_hvc_6[['total_ic_mou_6','total_ic_mou_7','total_ic_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['total_ic_mou_6','total_ic_mou_7','total_ic_mou_8']].median()

Inference-No observable trend, it is almost constant . 

### total_ic vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['total_ic_mou_6','total_ic_mou_7','total_ic_mou_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['total_ic_mou_6','total_ic_mou_7','total_ic_mou_8','churn']].groupby('churn').median()

Inference-We can see that the non churn customers have almost similar median through the period where as for churn customers it is almost 0 in the 8th month

## >> Derived column: difference from consecutive months

In [ ]:
telecom_hvc_6['diff_total_ic_6_7'] = telecom_hvc_6['total_ic_mou_6'] - telecom_hvc_6['total_ic_mou_7']
telecom_hvc_6['diff_total_ic_7_8'] = telecom_hvc_6['total_ic_mou_7'] - telecom_hvc_6['total_ic_mou_8']
telecom_hvc_6['diff_total_ic_6_8'] = telecom_hvc_6['total_ic_mou_6'] - telecom_hvc_6['total_ic_mou_8']

In [ ]:
telecom_hvc_6[['diff_total_ic_6_7','diff_total_ic_7_8','diff_total_ic_6_8','churn']].groupby('churn').mean()

Inference-We can see that for churn customers the total incoming calls seems to decrease consistently as we move to action phase

### Local incoming 

In [ ]:
telecom_hvc_6[['loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8']].median()

Inference - We can see No observable trend

### loc_ic vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['loc_ic_mou_6','loc_ic_mou_7','loc_ic_mou_8','churn']].groupby('churn').median()

We can see that the non churn customers have almost similar median through the period where as for churn customers it is almost 0 in the 8th month

### std incoming 

In [ ]:
telecom_hvc_6[['std_ic_mou_6','std_ic_mou_7','std_ic_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['std_ic_mou_6','std_ic_mou_7','std_ic_mou_8']].median()

No observable trend

### std_ic Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['std_ic_mou_6','std_ic_mou_7','std_ic_mou_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['std_ic_mou_6','std_ic_mou_7','std_ic_mou_8','churn']].groupby('churn').median()

We can see that the non churn customers have almost similar median through the period where as for churn customers it is almost 0 in the 8th month

### roam incoming 

In [ ]:
telecom_hvc_6[['roam_ic_mou_6','roam_ic_mou_7','roam_ic_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['roam_ic_mou_6','roam_ic_mou_7','roam_ic_mou_8']].mean()

### roam_og Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['roam_ic_mou_6','roam_ic_mou_7','roam_ic_mou_8']:
    plt.subplot(1,3,i)
    sns.violinplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['roam_ic_mou_6','roam_ic_mou_7','roam_ic_mou_8','churn']].groupby('churn').mean()

The customers who have churned have large amount of roaming incoming calls than the non churned customers

## Out going calls

In [ ]:
[col for col in telecom_hvc_6.columns if 'oc' in col]

### Total_outgoing calls

In [ ]:
telecom_hvc_6[['total_og_mou_6','total_og_mou_7','total_og_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['total_og_mou_6','total_og_mou_7','total_og_mou_8']].median()

In the first two months there is a slight decrease where as there is a sharp decrease from 7 th to 8th month

### total_og vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['total_og_mou_6','total_og_mou_7','total_og_mou_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['total_og_mou_6','total_og_mou_7','total_og_mou_8','churn']].groupby('churn').median()

We can see that in the intial month the median oc calls is similar, but it kept decreasing with very steep decent through the months.

## >> Derived column: difference from consecutive months

In [ ]:
telecom_hvc_6['diff_total_og_6_7'] = telecom_hvc_6['total_og_mou_6'] - telecom_hvc_6['total_og_mou_7']
telecom_hvc_6['diff_total_og_7_8'] = telecom_hvc_6['total_og_mou_7'] - telecom_hvc_6['total_og_mou_8']
telecom_hvc_6['diff_total_og_6_8'] = telecom_hvc_6['total_og_mou_6'] - telecom_hvc_6['total_og_mou_8']

In [ ]:
telecom_hvc_6[['diff_total_og_6_7','diff_total_og_7_8','diff_total_og_6_8','churn']].groupby('churn').mean()

Inference-For the churn customers the decrease in total out calls is very high, which says that the churn customers tend to make less number of calls in the action phase

### Local outgoing 

In [ ]:
telecom_hvc_6[['loc_og_mou_6','loc_og_mou_7','loc_og_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['loc_og_mou_6','loc_og_mou_7','loc_og_mou_8']].median()

Inference - We see in the intial two months the og calls are almost same where as there is a slight dip in the final month.

### loc_og vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['loc_og_mou_6','loc_og_mou_7','loc_og_mou_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['loc_og_mou_6','loc_og_mou_7','loc_og_mou_8','churn']].groupby('churn').median()

Inference-We can see that the non churn customers have almost similar median through the period where as for churn customers it is almost 0 in the 8th month and also they have significantly lower og calls when compared to non churn cutomers

In [ ]:
### roam outgoing 

In [ ]:
telecom_hvc_6[['roam_og_mou_6','roam_og_mou_7','roam_og_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

### std_og Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['std_og_mou_6','std_og_mou_7','std_og_mou_8']:
    plt.subplot(1,3,i)
    sns.boxplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['std_og_mou_6','std_og_mou_7','std_og_mou_8','churn']].groupby('churn').median()

Inference -We can see that the non churn customers have almost similar median through the period where as for churn customers it is almost 0 in the 8th month and also we can see that the decrease is very huge.

### roam outgoing 

In [ ]:
telecom_hvc_6[['roam_og_mou_6','roam_og_mou_7','roam_og_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_6[['roam_og_mou_6','roam_og_mou_7','roam_og_mou_8']].mean()

### roam_og Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['roam_og_mou_6','roam_og_mou_7','roam_og_mou_8']:
    plt.subplot(1,3,i)
    sns.violinplot(x = 'churn',y = col,data = telecom_hvc_6,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_6[['roam_og_mou_6','roam_og_mou_7','roam_og_mou_8','churn']].groupby('churn').mean()

Inference-The customers who have churned have large amount of std out going calls in the 6th month, but as with the progession their roam outgoing calls have also decreased.

In [ ]:
internet_cols = [col for col in telecom_hvc_6.columns if '_2g' in col or '_3g' in col]

In [ ]:
internet_cols

In [ ]:
telecom_hvc_7 = telecom_hvc_6.drop(internet_cols,axis = 1)

In [ ]:
telecom_hvc_7.info()

In [ ]:
telecom_hvc_7.columns.tolist()

#### 2. Univariate Analysis - Quantitative Variables

In [ ]:
def set_plotting_variable(figure_title, xlabel, ylabel):
    
    plt.title(figure_title)
    plt.xlabel(xlabel, labelpad = 15)
    plt.ylabel(ylabel, labelpad = 10)

In [ ]:
def plot_univariate(figsize_x, figsize_y, subplot_x, subplot_y, xlabel, ylabel, x_axis, data, wspace):
    
    plt.figure(figsize = (figsize_x, figsize_y))
    
    title_1 = "Distribution Plot of " + xlabel
    title_2 = "Box Plot of " + xlabel

    # Subplot - 1
    plt.subplot(subplot_x, subplot_y, 1)

    sns.distplot(data[x_axis], hist = True, kde = True, color = 'g')
    # Call Custom Function
    set_plotting_variable(title_1, xlabel, ylabel)

    # Subplot - 2
    plt.subplot(subplot_x, subplot_y, 2)

    sns.boxplot(x = x_axis, data = data, color = 'm')
    # Call Custom Function
    set_plotting_variable(title_2, xlabel, ylabel)
    
    plt.subplots_adjust(wspace = wspace)
    plt.show()

In [ ]:
# Univariate Plot Analysis of Quantitative Variables

category_list = ['night_pck_user_6', 'night_pck_user_7', 'night_pck_user_8', 
                 'fb_user_6', 'fb_user_7', 'fb_user_8', 'churn']
counter = 1

for col_list in telecom_hvc_7.columns:
    
    if col_list not in category_list:
        
        # Call Custom Function
        plot_univariate(figsize_x = 20,
                        figsize_y = 8,
                        subplot_x = 1,
                        subplot_y = 2,
                        xlabel = col_list,
                        ylabel = "Distribution",
                        x_axis = col_list,
                        data = telecom_hvc_7,
                        wspace = 0.2)

        counter += 1

Inference-There are lot of Outliers present in the variables. We will remove these outliers by performaing normalization in the later stage

## Arpu

In [ ]:
telecom_hvc_7[['arpu_6','arpu_7','arpu_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_7[['arpu_6','arpu_7','arpu_8']].mean()

### arpu Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['arpu_6','arpu_7','arpu_8']:
    plt.subplot(1,3,i)
    sns.violinplot(x = 'churn',y = col,data = telecom_hvc_7,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_7[['arpu_6','arpu_7','arpu_8','churn']].groupby('churn').mean()

Inference-The customers who have churned have large revenue in the 6th month, but as with the progession their revenue have also decreased.

## Onnet

In [ ]:
telecom_hvc_7[['onnet_mou_6','onnet_mou_7','onnet_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_7[['onnet_mou_6','onnet_mou_7','onnet_mou_8']].mean()

### onnet Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['onnet_mou_6','onnet_mou_7','onnet_mou_8']:
    plt.subplot(1,3,i)
    sns.violinplot(x = 'churn',y = col,data = telecom_hvc_7,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_7[['onnet_mou_6','onnet_mou_7','onnet_mou_8','churn']].groupby('churn').mean()

Inference-The customers who have churned has a decreasing onnnet calls per month as we move from 6 to 8th months.

## Offnet

In [ ]:
telecom_hvc_7[['offnet_mou_6','offnet_mou_7','offnet_mou_8']].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
telecom_hvc_7[['offnet_mou_6','offnet_mou_7','offnet_mou_8']].mean()

### offnet Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['offnet_mou_6','offnet_mou_7','offnet_mou_8']:
    plt.subplot(1,3,i)
    sns.violinplot(x = 'churn',y = col,data = telecom_hvc_7,showfliers = False)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_7[['offnet_mou_6','offnet_mou_7','offnet_mou_8','churn']].groupby('churn').mean()

Inference-The customers who have churned has a decreasing offnet calls per month as we move from 6 to 8th months.

## Is_rech

In [ ]:
telecom_hvc_7[['is_rech_6','is_rech_7','is_rech_8']].mean()

### is_rech Vs churn

In [ ]:
plt.figure(figsize = (15,5))
i = 1
for col in ['is_rech_6','is_rech_7','is_rech_8']:
    plt.subplot(1,3,i)
    sns.barplot(x = 'churn',y = col,data = telecom_hvc_7)
    i = i+1
plt.show()

In [ ]:
telecom_hvc_7[['is_rech_6','is_rech_7','is_rech_8','churn']].groupby('churn').mean()

Inference-The customers who have churned has stopped recharging their mobiles as the months progress from 6 to 8th which can be seen in decrease in is_rech values, which is a derived column from last_day_of_rech

# Correlation between variables

## ic variables

In [ ]:
ic_cols = [col for col in telecom_hvc_7.columns if '_ic' in col]
ic_cols.append('churn')

In [ ]:
plt.figure(figsize = (25,25))
sns.heatmap(telecom_hvc_7[ic_cols].corr(),annot = True)
plt.show()

Inference-We can see that there are quite a few number of variables correlated and this is expected because the total variables are obtained from the indiavidual variables. We can also see that there is a high correlation among corresponding variables for different months.We can also expect similar trend among other variables as well. Let the model handle these correlated variables.

## Correlation with Churn variable

In [ ]:
corr = telecom_hvc_7.corr()['churn'].sort_values(ascending = False)[1:]
plt.figure(figsize = (20,6))
plt.bar(range(len(corr)),corr)
plt.xticks(range(len(corr)),corr.index,rotation = 'vertical')
plt.show()

Inference-We can see that the drop of og calls and not recharging the mobiles in the 8th seems to signify that the customer is going to churn. We can also see that the std and roam variables are also contributing to the churn positively where as max rech and total calls seems to effect the churn negatively.

## Creating difference variables and averaging the first two months

In [ ]:
telecom_hvc_8 = telecom_hvc_7.drop(['total_rech_num_6','total_rech_num_7','total_rech_num_8','total_rech_amt_6','total_rech_amt_7','total_rech_amt_8','og_others_6','av_amt_6_7','total_amt_6_7'],axis = 1)

In [ ]:
telecom_hvc_8.drop([col for col in telecom_hvc_8.columns if 'diff_' in col],axis = 1,inplace = True)

In [ ]:
cols_to_excempt = ['is_rech_6','is_rech_7','is_rech_8']

In [ ]:
telecom_hvc_8.columns

In [ ]:
six_cols = []
seven_cols = []
for x in telecom_hvc_8.columns:
    if '_6' in x and x not in cols_to_excempt :
        six_cols.append(x)
    if '_7' in x and x not in cols_to_excempt :
        seven_cols.append(x)

In [ ]:
six_cols1 = [x.replace('_6','') for x in six_cols]

In [ ]:
telecom_hvc_9 = telecom_hvc_8.copy()

In [ ]:
for x in six_cols1:
    out = 'av_'+x+'_6_7'
    in_6 = x+'_6'
    in_7 = x+'_7'
    telecom_hvc_9[out] = (telecom_hvc_9[in_6]+telecom_hvc_9[in_7])/2

In [ ]:
for x in six_cols1:
    out1 = 'diff_'+x
    in_av = 'av_'+x+'_6_7'
    in_8 = x+'_8'
    telecom_hvc_9[out1] = telecom_hvc_9[in_av]-telecom_hvc_9[in_8]

In [ ]:
telecom_hvc_9.drop(six_cols+seven_cols,axis = 1,inplace = True)

In [ ]:
telecom_hvc_9.columns.tolist()

In [ ]:
telecom_hvc_9.info()

## Part 4 - Model Preparation

## 4.1 Handling class imbalance

In [ ]:
X = telecom_hvc_9.drop('churn',axis=1)
y = telecom_hvc_9.churn

In [ ]:
print(len(y))
print('\n')
print(y.value_counts())

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,test_size = 0.2,random_state = 28)

In [ ]:
# Generating a balanced X_train and y_train
smote = SMOTE(random_state = 10)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
print(len(y_train))
print('\n')
print(pd.Series(y_train).value_counts())

We can see that the class imbalance is handled after SMOTE

## 4.2 Test train split and Feature scaling

In [ ]:
index_list = []
for x in range(1,len(X_train)+1):
    index_list.append('I'+str(x))

In [ ]:
X_train = pd.DataFrame(X_train,columns = X.columns)

In [ ]:
X_train.index = index_list

In [ ]:
y_train.index = index_list
y_train.head()

In [ ]:
y_train.head()

In [ ]:
#X_train,X_test,y_train,y_test = train_test_split(X_balanced,y_balanced,train_size = 0.8,test_size = 0.2,stratify = y_balanced,random_state = 28)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [ ]:
len(X_train.index)

In [ ]:
X_train_scaled = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_train_scaled.index = X_train.index

In [ ]:
X_train_scaled.shape

In [ ]:
X_train.shape

In [ ]:
X_train_scaled.head()

In [ ]:
X_test_scaled = scaler.transform(X_test)

X_test_scaled = pd.DataFrame(X_test_scaled,columns = X_test.columns)
X_test_scaled.index = X_test.index

In [ ]:
X_test.shape

In [ ]:
X_test_scaled.shape

In [ ]:
X_test_scaled.head()

# Part 5 - Model Building

## 5.1 Logistic Regression model

In [ ]:
X_train_scaled.head()

In [ ]:
X_lr_1 = sm.add_constant(X_train_scaled)
logmodel1 = sm.GLM(y_train,X_lr_1,family = sm.families.Binomial())
logmodel1.fit().summary()

In [ ]:
y_train_pred = logmodel1.fit().predict(X_lr_1)

In [ ]:
y_train_pred.head()

In [ ]:
y_train.head()

In [ ]:
y_train_pred_final = y_train.reset_index().rename(columns = {0:'churn'})

In [ ]:
y_train_pred_final = y_train_pred_final.merge(y_train_pred.reset_index().rename(columns = {0:'churn_prob'}),on = 'index')

In [ ]:
y_train_pred_final.head()

### ROC curve

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.churn, y_train_pred_final.churn_prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.churn, y_train_pred_final.churn_prob)

# optimal cutoff point

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.churn_prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci','prec'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    prec = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci,prec]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
plt.figure(figsize = (10,5))
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.xticks(np.arange(0,0.95,0.05))
plt.show()

#### From the curve above, 0.54 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.churn_prob.map( lambda x: 1 if x > 0.55 else 0)

y_train_pred_final.head()

In [ ]:
confusion_sm = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final.final_predicted )
confusion_sm

In [ ]:
TP = confusion_sm[1,1] # true positive 
TN = confusion_sm[0,0] # true negatives
FP = confusion_sm[0,1] # false positives
FN = confusion_sm[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
print('Sensitivity:', (TP / float(TP+FN)))

# Let us calculate specificity
print('Specificity:',(TN / float(TN+FP)))
      
#precision
print('Precision:',(TP/float(FP+TP)))

## 4.2 PCA - Principal Component Analysis

In [ ]:
pca = PCA(svd_solver='randomized', random_state=28)
pca.fit(X_train_scaled)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
colnames = list(X_train_scaled.columns)
pcs_df = pd.DataFrame({'PC1':pca.components_[0],'PC2':pca.components_[1], 'Feature':colnames})
pcs_df.head(10)

In [ ]:
#Making the screeplot - plotting the cumulative variance against the number of components
%matplotlib inline
fig = plt.figure(figsize = (12,8))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.yticks(np.arange(0,1.05,0.05))
plt.show()

In [ ]:
pca_final = IncrementalPCA(n_components=20)

In [ ]:
telecom_train_pca = pca_final.fit_transform(X_train_scaled)
telecom_train_pca.shape

In [ ]:
pc_columns = []
for x in range(1,21):
    pc_columns.append('PC'+str(x))

In [ ]:
data_pc_df = pd.DataFrame((telecom_train_pca),columns = pc_columns)

In [ ]:
data_pc_df.head()

In [ ]:
data_pc_df.shape

In [ ]:
data_pc_df.index = X_train_scaled.index

## 4.3 Logistic regression with PCA

### model1

In [ ]:
X_pca_lr_1 = sm.add_constant(data_pc_df)
pca_lm1 = sm.GLM(y_train,X_pca_lr_1,family = sm.families.Binomial())
pca_lm1.fit().summary()

We can see that some of the P values are greater than 0.05.Hence dropping those columns.

In [ ]:
pca_data_df = data_pc_df.copy()

In [ ]:
pca_data_df.drop('PC20',axis = 1,inplace = True)

### model2

In [ ]:
X_pca_lr_2 = sm.add_constant(pca_data_df)
pca_lm2 = sm.GLM(y_train,X_pca_lr_2,family = sm.families.Binomial())
pca_lm2.fit().summary()

In [ ]:
pca_data_df.drop('PC17',axis = 1,inplace = True)

### model 3

In [ ]:
X_pca_lr_3 = sm.add_constant(pca_data_df)
pca_lm3 = sm.GLM(y_train,X_pca_lr_3,family = sm.families.Binomial())
pca_lm3.fit().summary()

In [ ]:
pca_data_df.drop('PC4',axis = 1,inplace = True)

### model 4

In [ ]:
X_pca_lr_4 = sm.add_constant(pca_data_df)
pca_lm4 = sm.GLM(y_train,X_pca_lr_4,family = sm.families.Binomial())
pca_lm4.fit().summary()

In [ ]:
pca_data_df.drop('PC19',axis = 1,inplace = True)

### model 5

In [ ]:
X_pca_lr_5 = sm.add_constant(pca_data_df)
pca_lm5 = sm.GLM(y_train,X_pca_lr_5,family = sm.families.Binomial())
pca_lm5.fit().summary()

all the p values in this model are less than 0.05 hence this is a stable model. VIF check is not needed because pca creates un correlated variables

### VIF

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_pca_lr_5.columns
vif['VIF'] = [variance_inflation_factor(X_pca_lr_5.values, i) for i in range(X_pca_lr_5.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
y_train_pred = pca_lm5.fit().predict(X_pca_lr_5)

In [ ]:
y_train_pred.head()

In [ ]:
y_train.head()

In [ ]:
y_train_pred_final = y_train.reset_index().rename(columns = {0:'churn'})

In [ ]:
y_train_pred_final = y_train_pred_final.merge(y_train_pred.reset_index().rename(columns = {0:'churn_prob'}),on = 'index')

In [ ]:
y_train_pred_final.head()

### ROC curve

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.churn, y_train_pred_final.churn_prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.churn, y_train_pred_final.churn_prob)

### optimal cutoff point

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.churn_prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci','prec'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    prec = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci,prec]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
plt.figure(figsize = (10,5))
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.xticks(np.arange(0,0.95,0.05))
plt.show()

#### From the curve above, 0.52 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.churn_prob.map( lambda x: 1 if x > 0.52 else 0)

y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.churn, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate 
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

### test scores

In [ ]:
telecom_test_pca = pca_final.transform(X_test_scaled)

test_pc_df = pd.DataFrame((telecom_test_pca),columns = pc_columns)

test_pc_df.index = X_test_scaled.index

In [ ]:
test_pc_df.head()

In [ ]:
X_test_pca_lr = sm.add_constant(test_pc_df[pca_data_df.columns.tolist()])

In [ ]:
y_test_pred = pca_lm5.fit().predict(X_test_pca_lr)

In [ ]:
y_test_pred.head()

In [ ]:
y_test.head()

In [ ]:
y_test_pred_final = y_test.reset_index().rename(columns = {0:'churn'})

In [ ]:
y_test_pred_final = y_test_pred_final.merge(y_test_pred.reset_index().rename(columns = {0:'churn_prob'}),on = 'mobile_number')

In [ ]:
y_test_pred_final.head()

In [ ]:
y_test_pred_final.head()

In [ ]:
y_test_pred_final['final_predicted'] = y_test_pred_final.churn_prob.map( lambda x: 1 if x > 0.54 else 0)

y_test_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_test_pred_final.churn, y_test_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_test_pred_final.churn, y_test_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate 
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

The train and test score are consistent therfore the model is not overfitting

## 4.4 Random forest with pca

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(data_pc_df,y_train)

In [ ]:
y_train_pred = rfc.predict(data_pc_df)

In [ ]:
y_train_pred

In [ ]:
print(classification_report(y_train,y_train_pred))

In [ ]:
# Printing confusion matrix
print(confusion_matrix(y_train,y_train_pred))

In [ ]:
print(accuracy_score(y_train,y_train_pred))

### test set

In [ ]:
y_test_pred = rfc.predict(test_pc_df)

In [ ]:
print(classification_report(y_test,y_test_pred))

In [ ]:
# Printing confusion matrix
print(confusion_matrix(y_test,y_test_pred))

In [ ]:
print(accuracy_score(y_test,y_test_pred))

## Hyper parameter tuning

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [5,10,15,20],
    'n_estimators': [500,1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1,verbose = 1)

In [ ]:
# Fit the grid search to the data
grid_search.fit(data_pc_df,y_train)

In [ ]:
# printing the optimal accuracy score and hyperparameters
print('We can get accuracy of',grid_search.best_score_,'using',grid_search.best_params_)

In [ ]:
rfc_final = RandomForestClassifier(n_estimators = 1000,max_depth = 20,bootstrap = True,random_state = 28)

In [ ]:
rfc_final.fit(data_pc_df,y_train)

In [ ]:
y_train_pred = rfc_final.predict(data_pc_df)

In [ ]:
print(classification_report(y_train,y_train_pred))

In [ ]:
# Printing confusion matrix
print(confusion_matrix(y_train,y_train_pred))

In [ ]:
print(accuracy_score(y_train,y_train_pred))

### test

In [ ]:
y_test_pred = rfc_final.predict(test_pc_df)

In [ ]:
print(classification_report(y_test,y_test_pred))

In [ ]:
# Printing confusion matrix
print(confusion_matrix(y_test,y_test_pred))

In [ ]:
print(accuracy_score(y_test,y_test_pred))

## 4.5 Factors contributing to Churn

### Feature selection using RFE

In [ ]:
lr_rfe = LogisticRegression()

In [ ]:
rfe20 = RFE(lr_rfe, 20)             
rfe20 = rfe20.fit(X_train_scaled, y_train)

In [ ]:
print('Selected columns from RFE:')
print(X_train_scaled.columns[rfe20.support_].tolist())

In [ ]:
cols = X_train_scaled.columns[rfe20.support_].tolist()

In [ ]:
cols

### Model creation with 20 features

In [ ]:
X_model20 = sm.add_constant(X_train_scaled[cols])
logmodel20 = sm.GLM(y_train,X_model20,family = sm.families.Binomial())
logmodel20.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('diff_loc_og_mou')

### Model with 19 features

In [ ]:
X_model19 = sm.add_constant(X_train_scaled[cols])
logmodel19 = sm.GLM(y_train,X_model19,family = sm.families.Binomial())
logmodel19.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('diff_arpu')

## Model with 18 features

In [ ]:
X_model18 = sm.add_constant(X_train_scaled[cols])
logmodel18 = sm.GLM(y_train,X_model18,family = sm.families.Binomial())
logmodel18.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('diff_total_ic_mou')

### Model with 17 features

In [ ]:
X_model17 = sm.add_constant(X_train_scaled[cols])
logmodel17 = sm.GLM(y_train,X_model17,family = sm.families.Binomial())
logmodel17.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('diff_total_og_mou')

### Model with 16 Features 

In [ ]:
X_model16 = sm.add_constant(X_train_scaled[cols])
logmodel16 = sm.GLM(y_train,X_model16,family = sm.families.Binomial())
logmodel16.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('av_arpu_6_7')

### Model with 15 Features

In [ ]:
X_model15 = sm.add_constant(X_train_scaled[cols])
logmodel15 = sm.GLM(y_train,X_model15,family = sm.families.Binomial())
logmodel15.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('loc_ic_mou_8')

### Model with 14 features

In [ ]:
X_model14 = sm.add_constant(X_train_scaled[cols])
logmodel14 = sm.GLM(y_train,X_model14,family = sm.families.Binomial())
logmodel14.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('av_loc_og_mou_6_7')

### Model with 13 Features

In [ ]:
X_model13 = sm.add_constant(X_train_scaled[cols])
logmodel13 = sm.GLM(y_train,X_model13,family = sm.families.Binomial())
logmodel13.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
cols.remove('loc_og_t2m_mou_8')

In [ ]:
X_model11 = sm.add_constant(X_train_scaled[cols])
logmodel11 = sm.GLM(y_train,X_model11,family = sm.families.Binomial())
logmodel11.fit().summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_scaled[cols].columns
vif['VIF'] = [variance_inflation_factor(X_train_scaled[cols].values, i) for i in range(X_train_scaled[cols].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

All p values are less than 0.05 and all VIFs are less than 5. Therefore this is stable model.

In [ ]:
y_train_pred = logmodel11.fit().predict(X_model11)

In [ ]:
y_train_pred.head()

In [ ]:
y_train_pred_final = y_train.reset_index().rename(columns = {0:'churn'})

In [ ]:
y_train_pred_final = y_train_pred_final.merge(y_train_pred.reset_index().rename(columns = {0:'churn_prob'}),on = 'index')

In [ ]:
y_train_pred_final.head()

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.churn, y_train_pred_final.churn_prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.churn, y_train_pred_final.churn_prob)

### optimal cutoff point

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.churn_prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci','prec'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    prec = cm1[1,1]/(cm1[1,1]+cm1[0,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci,prec]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
plt.figure(figsize = (10,5))
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.xticks(np.arange(0,0.95,0.05))
plt.show()

#### From the curve above, 0.54 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.churn_prob.map( lambda x: 1 if x > 0.61 else 0)

y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.churn, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.churn, y_train_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate 
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

### test scores

In [ ]:
y_test_pred = logmodel11.fit().predict(sm.add_constant(X_test_scaled[cols]))

In [ ]:
y_test_pred.head()

In [ ]:
y_test.head()

In [ ]:
y_test_pred_final = y_test.reset_index().rename(columns = {0:'churn'})

In [ ]:
y_test_pred_final = y_test_pred_final.merge(y_test_pred.reset_index().rename(columns = {0:'churn_prob'}),on = 'mobile_number')

In [ ]:
y_test_pred_final.head()

In [ ]:
y_test_pred_final['final_predicted'] = y_test_pred_final.churn_prob.map( lambda x: 1 if x > 0.54 else 0)

y_test_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_test_pred_final.churn, y_test_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_test_pred_final.churn, y_test_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate 
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

The train and test score are consistent therfore the model is not overfitting

## Final equation

In [ ]:
print(logmodel11.fit().summary())

__churn = -7.1867    +16.8248\*`roam_og_mou_8` -19.6774 \*`loc_og_t2f_mou_8 ` -54.5052\*`loc_og_mou_8 ` -5.6667  \*`total_og_mou_8 ` -24.6461\*`total_ic_mou_8` -7.5832 \*`spl_ic_mou_8` -21.8440 \*`last_day_rch_amt_8`   -14.8813 \*`total_rech_count_8` + 9.6326 \*`av_roam_ic_mou_6_7` +  3.0978 \*`av_std_og_t2m_mou_6_7` + 18.3178\*`diff_loc_ic_mou`__

# Conclusions

- Random forest with PCA has better prediction capability than the logistic regression with accuracy of 0.94 and 0.95 sensitivity
- `loc_og_t2f_mou_8 `,`total_ic_mou_8`,`last_day_rch_amt_8`,`spl_ic_mou_8`,`total_rech_count_8 `,`roam_og_mou_8` are effecting the churn negatively decreasing the probability of churn and `diff_loc_ic_mou`,`av_roam_ic_mou_6_7`,`av_loc_og_t2m_mou_6_7` are effecting the churn positively, increasing the probability of churn.
- From this we can say that std and roam variables are causing the churn, which may mean that competition are offering better rates, which may be resulting in churn

# Recommendations

        - We can lower the roaming charges as this is one of the reasons for pre-paid customers to seek out local operators when on roaming.
        -Also customize pre-paid plan offers can be issued: For example customized plans based on where customer minutes of usage are higher - for instance outgoing/std/t2t/t2m etc .  
        -For customers who are staying longer with network we can increase customer loyalty points and offer some special discounts.  